In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/data_fusion_little.csv')
data = data.dropna()
data.head()

,Titre,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores,Id,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
8,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,ARI272XF8TOL4,Christopher J. Bray,74/81,5.0,955411200,Ecclesiological Milestone,With the publication of Everett Ferguson's boo...
9,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,A36TPZSH8LBT1,haskell,2/3,5.0,1311465600,Early Christian development of the Church,Everett Ferguson approaches the subject of ear...
57,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A30IUAABSEHEPI,Lawrence Duckles,9/9,3.0,1055980800,The Battleship Bismarck reviewed,This book is both a history and a photo album ...
58,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A29SCEAL3JPMYW,Tim Grasshoff,0/0,4.0,1217808000,nice historical pictures,I like this book first one I've seen with so m...
74,Beginner's Yoruba (Hippocrene Beginner's Series),"""Beginner's Yoruba"" is now available with two ...",['Kayode J. Fakinlede'],http://books.google.com/books/content?id=xLe4n...,http://books.google.nl/books?id=xLe4nWzeSw0C&p...,Hippocrene Books,2005,http://books.google.nl/books?id=xLe4nWzeSw0C&d...,['Foreign Language Study'],1.0,0781810698,19.77,A1F0EV2MBF208I,"Olena Y. Rabinowitz ""Book lover""",14/14,4.0,1156291200,Very authentic,This is my first encounter with Yoruba and I h...


In [3]:
from sklearn.model_selection import train_test_split

X_polarite = data[['Titre', 'revue/texte']]
Y_polarite = data['revue/score']

In [4]:
from sklearn.preprocessing import OrdinalEncoder

def encode_simple(X_polarite, Y_polarite):

    X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(X_polarite, Y_polarite, test_size=0.2, random_state=42)

    # Initialiser l'encodeur avec gestion des valeurs inconnues
    titre_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
    texte_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    # Encodage des données d'entraînement
    X_train_polarite[['Titre']] = titre_encoder.fit_transform(X_train_polarite[['Titre']])
    X_train_polarite[['revue/texte']] = texte_encoder.fit_transform(X_train_polarite[['revue/texte']])

    # Encodage des données de test
    X_test_polarite[['Titre']] = titre_encoder.transform(X_test_polarite[['Titre']])
    X_test_polarite[['revue/texte']] = texte_encoder.transform(X_test_polarite[['revue/texte']])

    return X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

def encode_tfidf(X_polarite, Y_polarite):

    X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(X_polarite, Y_polarite, test_size=0.2, random_state=42)

    # Initialiser le vectorizer TF-IDF
    tfidf_vectorizer = TfidfVectorizer()

    # Encodage des données d'entraînement
    X_train_polarite_titre_tfidf = tfidf_vectorizer.fit_transform(X_train_polarite['Titre'])
    X_train_polarite_texte_tfidf = tfidf_vectorizer.fit_transform(X_train_polarite['revue/texte'])

    # Encodage des données de test
    X_test_polarite_titre_tfidf = tfidf_vectorizer.transform(X_test_polarite['Titre'])
    X_test_polarite_texte_tfidf = tfidf_vectorizer.transform(X_test_polarite['revue/texte'])

    # Combiner les deux colonnes encodées
    X_train_polarite = hstack([X_train_polarite_titre_tfidf, X_train_polarite_texte_tfidf])
    X_test_polarite = hstack([X_test_polarite_titre_tfidf, X_test_polarite_texte_tfidf])

    return X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_simple(X_polarite, Y_polarite)

# Initialisation du modèle Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.01, max_depth=8, random_state=42)

# Entraînement
gb_model.fit(X_train_polarite, Y_train_polarite)

# Prédiction
Y_pred_gb = gb_model.predict(X_test_polarite)

# Enregistrement du model
import pickle
filename = 'models2/regression_model_simple.pkl'
pickle.dump(gb_model, open(filename, 'wb'))

# Évaluation
mse_gb = mean_squared_error(Y_test_polarite, Y_pred_gb)
print(f"📊 MSE avec Gradient Boosting : {mse_gb:.4f}")

📊 MSE avec Gradient Boosting : 1.6866


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_tfidf(X_polarite, Y_polarite)

# Initialisation du modèle Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.01, max_depth=8, random_state=42)

# Entraînement
gb_model.fit(X_train_polarite, Y_train_polarite)

# Prédiction
Y_pred_gb = gb_model.predict(X_test_polarite)

# Enregistrement du model
import pickle
filename = 'models2/regression_model_tfidf.pkl'
pickle.dump(gb_model, open(filename, 'wb'))

# Évaluation
mse_gb = mean_squared_error(Y_test_polarite, Y_pred_gb)
print(f"📊 MSE avec Gradient Boosting : {mse_gb:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_simple(X_polarite, Y_polarite)

# Initialisation du modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=5000, max_depth=10, random_state=42)

# Entraînement
rf_model.fit(X_train_polarite, Y_train_polarite)

# Prédiction
Y_pred_rf = rf_model.predict(X_test_polarite)

# Enregistrement du model
import pickle
filename = 'models2/classification_model_simple.pkl'
pickle.dump(rf_model, open(filename, 'wb'))

# Évaluation
accuracy_rf = accuracy_score(Y_test_polarite, Y_pred_rf)
print(f"📊 Accuracy avec Random Forest : {accuracy_rf:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = encode_tfidf(X_polarite, Y_polarite)

# Initialisation du modèle Random Forest
rf_model = RandomForestClassifier(n_estimators=5000, max_depth=10, random_state=42)

# Entraînement
rf_model.fit(X_train_polarite, Y_train_polarite)

# Prédiction
Y_pred_rf = rf_model.predict(X_test_polarite)

# Enregistrement du model
import pickle
filename = 'models2/classification_model_tfidf.pkl'
pickle.dump(rf_model, open(filename, 'wb'))

# Évaluation
accuracy_rf = accuracy_score(Y_test_polarite, Y_pred_rf)
print(f"📊 Accuracy avec Random Forest : {accuracy_rf:.4f}")